In [1]:
import random
import math

def euclidean_distance(point_a, point_b):
    return math.dist(point_a, point_b)

def compute_distance_matrix(cities):
    n = len(cities)
    return [[euclidean_distance(cities[i], cities[j]) for j in range(n)] for i in range(n)]

def build_greedy_tour_with_tumbling(
    start_city, distance_matrix, city_nutrients,
    chemotaxis_sample_size, tumble_probability=0.15, initial_health=100.0
):
    """
    Constructs a TSP tour using a greedy approach plus random 'tumble' moves.
    At each step, with probability tumble_probability, selects a random next city (tumble),
    otherwise acts greedily among a chemotactic sample.
    """
    n_cities = len(city_nutrients)
    tour = [start_city]
    visited = set(tour)
    current_city = start_city
    total_length = 0.0
    health = initial_health

    cities_to_visit = n_cities - 1
    for step in range(cities_to_visit):
        candidates = [city for city in range(n_cities) if city not in visited]
        if not candidates:
            break

        # With probability tumble_probability, do a tumble (random next city)
        if random.random() < tumble_probability:
            next_city = random.choice(candidates)
        else:
            # Chemotactic sensing: sample random subset, move to nearest candidate
            sample_size = min(chemotaxis_sample_size, len(candidates))
            sampled_candidates = random.sample(candidates, sample_size)
            next_city = min(
                sampled_candidates, key=lambda c: distance_matrix[current_city][c]
            )

        segment_length = distance_matrix[current_city][next_city]

        # Update state
        health -= segment_length
        health += city_nutrients[next_city]
        total_length += segment_length
        tour.append(next_city)
        visited.add(next_city)
        current_city = next_city

    # Close tour to return to start
    segment_length = distance_matrix[current_city][tour[0]]
    total_length += segment_length
    health -= segment_length
    tour.append(tour[0])

    return tour, total_length, health

def bfoa_tsp(
    city_coordinates,
    n_bacteria=30,
    chemotaxis_sample_size=5,
    reproduction_cycles=4,
    elimination_cycles=2,
    elimination_probability=0.1,
    alpha=1.0,
    tumble_probability=0.15
):
    """
    Bacterial Foraging Optimization Algorithm for the TSP.
    - Each bacterium is a tour-building strategy from a start city.
    - Greedy chemotaxis with random tumbling.
    - Bacteria reproduce, and some are eliminated/dispersed.
    """
    n_cities = len(city_coordinates)
    distance_mat = compute_distance_matrix(city_coordinates)

    city_nutrients = [10.0 for _ in range(n_cities)]

    # Initialize population: each bacterium starts from a (possibly random) city
    initial_start_city = 0
    bacteria_population = [initial_start_city for _ in range(n_bacteria)]

    best_tour = None
    best_fitness = float('inf')

    for elim_cycle in range(elimination_cycles):
        for repro_cycle in range(reproduction_cycles):
            bacteria_statistics = []
            for bacterium_index, start_city in enumerate(bacteria_population):
                tour, tour_length, health = build_greedy_tour_with_tumbling(
                    start_city, distance_mat, city_nutrients,
                    chemotaxis_sample_size, tumble_probability
                )

                # Fitness combines distance and (optionally) remaining health
                fitness = tour_length - alpha * health
                bacteria_statistics.append((fitness, tour, tour_length, health))

                if fitness < best_fitness:
                    best_tour, best_fitness = tour, fitness

            # Reproduction: select the top 50% by fitness
            bacteria_statistics.sort(key=lambda x: x[0])  # Lower fitness better
            survivors = bacteria_statistics[: n_bacteria // 2]
            # Repopulate by randomly choosing from the survivors
            bacteria_population = [
                random.choice(survivors)[1][0] for _ in range(n_bacteria)
            ]

        # Elimination-dispersal: randomly re-seed starting cities for some bacteria
        for i, bacterium in enumerate(bacteria_population):
            if random.random() < elimination_probability:
                bacteria_population[i] = random.randint(0, n_cities - 1)

    return best_tour, best_fitness


In [15]:


    # 10 cities with random (x, y) coordinates
#cities = [(random.uniform(0,100), random.uniform(0,100)) for _ in range(10)]
    #15 cities with predetermined coordinates
cities = [
        (10, 10), (20, 30), (30, 10), (40, 25), (50, 5),
        (60, 30), (70, 10), (80, 20), (90, 5), (25, 60),
       # (35, 45), (55, 55), (65, 65), (75, 45), (85, 60)
    ]
best_tour, best_fitness = bfoa_tsp(
        city_coordinates=cities,
        n_bacteria=30,
        chemotaxis_sample_size=4,
        reproduction_cycles=4,
        elimination_cycles=2,
        elimination_probability=0.1,
        alpha=1.0,
        tumble_probability=0.15
    )
print("Best tour found:", best_tour)
print("Tour length minus health-weighted fitness:", best_fitness)


Best tour found: [0, 2, 3, 4, 6, 8, 7, 5, 9, 1, 0]
Tour length minus health-weighted fitness: 291.76028654753145
